# Python赋值机制

先看一个例子:

In [1]:
x = [1, 2, 3]
y = x
x[1] = 100
print y

[1, 100, 3]


改变变量x的值,变量y的值也随着改变,这与Python内部的赋值机制有关.

## 简单类型

先来看这一段代码在Python中的执行过程.

    x = 500
    y = x
    y = 'foo'

    x = 500  
Python分配了一个pyint大小的内存pos1用来储存对象500,然后,Python在命名空间中让变量x指向了这一块内存,注意,整数是不可变类型,所以这块内存的内容是不可变的.

|内存|命名空间|
|----|----|
|pos1 : Pyint(500)(不可变)|x:pos1|

    y = x
Python并没有使用新的内存来储存变量y的值,而是在命名空间中,让变量y与变量x指向了同一块内存空间

|内存|命名空间|
|---|---|
|pos1:Pyint(500)(不可变)|x:pos1  y:pos1|

    y = 'foo'
Python此时分配一个PyStr大小的内存pos2来储存对象foo,然后改变变量y所指的对象.    

|内存|命名空间|
|---|---|
|pos1:Pyint(500)(不可变)|x:pos1|
|pos2:PyStr('foo")(不可变)|y:pos2|

对这一过程进行验证,可以使用id函数.

    id(x)

返回变量x的内存地址.

In [2]:
x = 500
id(x)

43934752

In [3]:
y = x
id(y)

43934752

也可以使用is来判断是不是指向同一个事物:

In [4]:
x is y

True

现在y指向另一块内存:

In [5]:
y = 'foo'
id(y)

43182632

In [6]:
x is y

False

Python会为每个出现的对象进行赋值,哪怕它们的值是一样的,例如:

In [7]:
x = 500
id(x)

43933220

In [8]:
y = 500
id(y)

43934752

In [9]:
x is y

False

不过,为了提高内存利用效率,对于一些简单的对象,如一些数值较小的int对象,Python采用了重用对象内存的方法

In [10]:
x = 2
id(x)

35257652

In [11]:
y = 2
id(y)

35257652

In [12]:
x is y

True

## 容器类型

现在来看另一段代码:  
    x = [500, 501, 502]  
    y = x  
    y[1] = 600  
    y = [700, 800]

x = [500, 501, 502]


Python为3个pyint分配内存pos1,pos2,pos3(不可变),然后为列表分配内存pos4,它包含3个位置,分别指向这3个内存,最后再让变量x指向这个列表.

|内存|命名空间|
|---|---|
|pos1:PyInt(500)(不可变)|x:pos4|
|pos2:PyInt(501)(不可变)|
|pos3:PyInt(502)(不可变)|
|pos4:PyList(pos1,pos2,pos3)(可变)|

y = x  
并没有创建新的对象,只需要将y指向pos4即可.

|内存|命名空间|
|---|---|
|pos1:PyInt(500)(不可变)|x:pos4|
|pos2:PyInt(501)(不可变)|y:pos4|
|pos3:PyInt(502)(不可变)|
|pos4:PyList(pos1,pos2,pos3)(可变)|

y[1] = 600  
原来y[1]这个位置指向的是pos2,由于不能修改pos2的值,所以首先为600分配新内存pos5.  
再把y[1]指向的位置修改为pos5.此时,由于pos2位置的对象已经没有用了,Python会自动调用垃圾处理机制将它回收.

|内存|命名空间|
|---|---|
|pos1:PyInt(500)(不可变)|x:pos4|
|pos2:垃圾回收|y:pos4|
|pos3:PyInt(502)(不可变)|
|pos4:PyList(pos1,pos5,pos3)(可变)|
|pos5:PyInt(600)(不可变)|

y = [700, 800]  
首先创建这个列表,然后将变量y指向它.

|内存|命名空间|
|---|---|
|pos1:PyInt(500)(不可变)|x:pos4|
|pos3:PyInt(502)(不可变)|
|pos4:PyList(pos1,pos5,pos3)(可变)|
|pos5:PyInt(600)(不可变)|
|pos6:PyInt(700)(不可变)|
|pos7:PyInt(800)(不可变)|
|pos8:PyList(pos6, pos7)(可变)|y:pos8|

对这一过程进行验证:

In [13]:
x = [500, 501, 502]
print id(x[0])
print id(x[1])
print id(x[2])
print id(x)

43931172
43933220
50453084
61631344


赋值,id(y)与id(x)相同.

In [14]:
y = x
id(y)

61631344

In [15]:
x is y

True

修改y[1], id(y)并不改变.

In [16]:
y[1] = 600
print id(y)

61631344


id(x[1])和id(y[1])的值改变了

In [17]:
print id(x[1])
print id(y[1])

50451252
50451252


改变y的值,id(y)的值改变

In [18]:
y = [700, 800]
print id(y)
print id(x)

62620336
61631344
